In [1]:
# !uv pip install langchain langchain-openai langchain-community faiss-cpu openai tiktoken trafilatura playwright nest_asyncio langchain_experimental
# !uv add langchain langchain-openai langchain-community faiss-cpu openai tiktoken trafilatura playwright nest_asyncio langchain_experimental
# !playwright install

In [6]:
from src.components.ingestion import DataIngestionPipeline 
 
ingestion = DataIngestionPipeline(model_config_path="configs/model_config.yaml", environment_config_path="config.yaml")
vectorstore = ingestion.get_or_create_vectorstore( 
    use_semantic_chunking=True,   
) 

# res = vectorstore.similarity_search(
#     query="มีอะไรบ้าง?",
#     search_type="similarity",
#     search_kwargs={"k": 4}  # จำนวน chunks ที่จะดึงมา
# ) 

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # จำนวน chunks ที่จะดึงมา
)

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import os
from src.utils import get_config

config = get_config() 
 
# กำหนด LLM เป็น Typhoon
typhoon_llm = ChatOpenAI(
    api_key=config['typhoon']['token'],
    base_url="https://api.opentyphoon.ai/v1",
    model="typhoon-v2.1-12b-instruct",
    temperature=0
)
 

template = """
### ROLE ###
คุณคือ AI Sales & Support Specialist ของบริษัท Investic

### OBJECTIVE ###
เป้าหมายของคุณคือการแปลงคำถามของลูกค้าให้กลายเป็นการขาย (Conversion) โดยการให้ข้อมูลที่ถูกต้อง, เน้นคุณประโยชน์, และสร้างประสบการณ์ที่น่าประทับใจ โดยต้องอ้างอิงจากข้อมูลในส่วน KNOWLEDGE BASE เท่านั้น

### TONE OF VOICE ###
- Professional (เป็นมืออาชีพ)
- Helpful (ช่วยเหลือ)
- Persuasive (โน้มน้าว)
- Confident (มั่นใจ)
- Thai language, formal "ครับ" tone.

### WORKFLOW ###
1.  **Acknowledge and Understand:** ขึ้นต้นด้วยการตอบรับคำถามของลูกค้า และแสดงความเข้าใจในสิ่งที่เขาต้องการ
2.  **Extract & Reframe:** ดึงข้อมูลที่เกี่ยวข้องจาก KNOWLEDGE BASE จากนั้นนำเสนอในรูปแบบของ "ประโยชน์" ที่ลูกค้าจะได้รับ
3.  **Address Concerns & Compare:** หากลูกค้าแสดงความกังวลหรือต้องการเปรียบเทียบ ให้ใช้ข้อมูลเพื่อคลายความกังวลและชี้ให้เห็นจุดเด่นของสินค้าเรา
4.  **Suggest & Guide (Call-to-Action):** เสนอขั้นตอนต่อไปอย่างชัดเจน เช่น แนะนำให้ซื้อ, สอบถามข้อมูลการชำระเงิน, หรือให้ข้อมูลโปรโมชั่น

### RULES ###
- DO NOT invent information not present in the KNOWLEDGE BASE.
- If information is missing, politely state that you need to check with a specialist and offer to assist with other questions.
- Always end the conversation with a call to action or an offer for further help.

---
### KNOWLEDGE BASE (Context) ###
{context}

### CUSTOMER QUESTION (Question) ###
{question}

---
### RESPONSE ###
"""
prompt = ChatPromptTemplate.from_template(template)
 

# 4. สร้าง Chain (RAG Chain)
# คือการต่อท่อการทำงานทั้งหมดเข้าด้วยกัน
rag_chain = (
    RunnableParallel(
        context=retriever,
        question=RunnablePassthrough()
    )
    | prompt
    | typhoon_llm
    | StrOutputParser()
)

# 5. ลองถามคำถามผ่าน Chain
print("\n--- 🤖 เริ่มระบบถาม-ตอบ (RAG) ---")
question_to_ask = "มีเวิคช้อบอะไรบ้างครับ เเล้วขายยังไง"
answer = rag_chain.invoke(question_to_ask)

print(f"คำถาม: {question_to_ask}")
print(f"คำตอบจาก AI: {answer}")


--- 🤖 เริ่มระบบถาม-ตอบ (RAG) ---
คำถาม: มีเวิคช้อบอะไรบ้างครับ เเล้วขายยังไง
คำตอบจาก AI: สวัสดีครับ ยินดีต้อนรับสู่ SCB 10X ครับ

สำหรับเวิร์คช็อปที่เรามีนะครับ ตอนนี้เรามี "Online Workshops จับมือเทรดด้วย Quant Data" ซึ่งถ้าคุณซื้อ 1 เวิร์คช็อปขึ้นไป คุณจะได้รับสิทธิพิเศษมากมายเลยครับ เช่น

*   RERUN ทุก Session ทั้ง Stage 1 & 2 (รวม 11 ชั่วโมง)
*   RERUN Crypto Quant Trading (รวม 7 ชั่วโมง)
*   สิทธิ์ใช้เครื่องมือดู Quant Tool ของ Investic Analytics Studio เป็นระยะเวลา 1 เดือน

นอกจากนี้ ยังมี "Rerun Quant Offside 3x" ที่จะพาคุณเจาะลึกเรื่องการใช้ Market Breadth ในการ Filter ตลาด, การตรวจดูพฤติกรรมหุ้นที่แข็งกว่าตลาด, การวิเคราะห์สถิติตลาดหุ้นในอดีต, Use case การพัฒนาระบบเทรดในตลาดไทย, การประเมินมูลค่าหุ้นด้วย Fundamental Analysis และอื่นๆ อีกมากมายครับ

สำหรับการซื้อ สามารถทำได้ง่ายๆ เลยครับ เพียงเข้าไปที่เว็บไซต์ของเรา [https://www.scb10x.com/](https://www.scb10x.com/) แล้วเลือกเวิร์คช็อปที่สนใจ หลังจากนั้นก็ทำตามขั้นตอนการชำระเงินได้เลยครับ

หากคุณมีคำถามเพิ่มเติม หรือต้องการข้อ

In [10]:
# https://deepeval.com/docs/evaluation-datasets

import os
from pathlib import Path
from deepeval.dataset import EvaluationDataset
from deepeval.synthesizer import Synthesizer
from src.utils.app_config import AppConfig

# --- Setup ---
# โหลด config และตั้งค่า API Key (เหมือนเดิม)
cfg = AppConfig.from_files("configs/model_config.yaml", "config.yaml")
os.environ.setdefault("OPENAI_API_KEY", cfg.openai_token)

# --- Generation --- 
synthesizer = Synthesizer(model="gpt-4.1")

dataset = EvaluationDataset()
 

# 2. เรียกใช้เมธอดเพื่ออ่านไฟล์และสร้าง test case ภายในอ็อบเจ็กต์
# DeepEval จะอ่านไฟล์, แบ่งเนื้อหา (chunking), แล้วส่งให้ LLM สร้างคำถาม-คำตอบ เลยต้องลง chromadb
dataset.generate_goldens_from_docs(
    document_paths=["data/raw/overall.txt"]
)

print(f"สร้างชุดข้อมูลทดสอบเสร็จสิ้น จำนวน {len(dataset)} เคส")
 

[Confident AI Synthesizer Log] WARNING: Filtering not applied: Not enough chunks in smallest document

[Confident AI Synthesizer Log] SUCCESS: Context Construction: Utilizing 6 out of 7 chunks.

สร้างชุดข้อมูลทดสอบเสร็จสิ้น จำนวน 0 เคส


In [13]:
dataset.goldens

[Golden(input='How does the TFEX workshop utilize Bollinger Bands for trend reversal and a 100% win rate?', actual_output=None, expected_output='The TFEX workshop utilizes Bollinger Bands to identify trend reversal points and achieve a 100% win rate by teaching participants to accurately catch the market bottom. The strategy involves using Bollinger Band Market Breadth to pinpoint buy and sell signals and has been demonstrated through practical examples, such as during the 2011 flood crisis and other major market events.', context=['ฟรีเครื่องมือ Investic Analytics Studio เต็มๆ 1 เดือน!\nOnline Workshops\nออกแบบมาเพื่อให้คุณได้ลงมือทำจริง พร้อมเทคนิคและสิทธิ์ได้เครื่องมือที่ใช้งานตลอด 1 เดือน เพื่อลุยเทรดในตลาดกันจริงๆ\n\n**Workshop Details**\n\n- **Online Workshops "จับมือเทรดด้วย Quant Data"**\n  - เรียนรู้แบบเจาะลึกกับเหล่า Guru Quant พร้อมใช้ฟรีเครื่องมือ Investic Analytics Studio เต็มๆ 1 เดือน!\n- **TFEX - Win rate 100% จับ Panic Sell ด้วย Bollinger Band Market Breadth**\n  - เรีย

In [6]:
import os
import json
from deepeval.dataset import EvaluationDataset
from deepeval.synthesizer import Synthesizer
from src.utils.app_config import AppConfig
from openai import OpenAI

# --- Setup ---
cfg = AppConfig.from_files("configs/model_config.yaml", "config.yaml")
os.environ.setdefault("OPENAI_API_KEY", cfg.openai_token)
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# ===================================================================
# ขั้นตอนที่ 1: สร้างชุดข้อมูลเป็นภาษาอังกฤษจากหลายไฟล์
# ===================================================================
print("ขั้นตอนที่ 1: กำลังสร้างชุดข้อมูลเป็นภาษาอังกฤษจากหลายไฟล์...")
english_synthesizer = Synthesizer(model="gpt-4o")
dataset_en = EvaluationDataset()

# ระบุ path ของไฟล์ทั้งหมดที่ต้องการให้ Synthesizer ใช้
document_paths_for_synthesis = [
    "data/raw/overall.txt",
    "data/raw/rerun.txt",
    "data/raw/workshop.txt"
]

dataset_en.generate_goldens_from_docs(
    document_paths=document_paths_for_synthesis, # <<< ตรงนี้แหละครับ
    synthesizer=english_synthesizer
)
print(f"สร้างชุดข้อมูลภาษาอังกฤษเสร็จสิ้น จำนวน {len(dataset_en.goldens)} เคส")


# ===================================================================
# ขั้นตอนที่ 2: แปลแต่ละ Test Case เป็นภาษาไทย
# (โค้ดส่วนนี้เหมือนเดิมจากตัวอย่างก่อนหน้า)
# ===================================================================
print("\nขั้นตอนที่ 2: กำลังแปลชุดข้อมูลเป็นภาษาไทย...")
translated_goldens = []
for i, golden in enumerate(dataset_en.goldens):
    print(f"  -> กำลังแปลเคสที่ {i+1}/{len(dataset_en.goldens)}...")
    try:
        prompt_to_translate = f"""
        Translate the following question and answer into Thai.
        Provide the output as a JSON object with two keys: "translated_question" and "translated_answer".

        Question: "{golden.input}"
        Answer: "{golden.expected_output}"
        """

        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt_to_translate}],
            response_format={"type": "json_object"},
            temperature=0.0
        )

        translated_json = json.loads(response.choices[0].message.content)

        golden.input = translated_json.get("translated_question", golden.input)
        golden.expected_output = translated_json.get("translated_answer", golden.expected_output)

        translated_data = {
            "input": golden.input,
            "expected_output": golden.expected_output,
            "retrieval_context": golden.retrieval_context
        }
        translated_goldens.append(translated_data)

    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการแปลเคสที่ {i+1}: {e}")
        original_data = {
            "input": golden.input,
            "expected_output": golden.expected_output,
            "retrieval_context": golden.retrieval_context
        }
        translated_goldens.append(original_data)

print(f"แปลชุดข้อมูลภาษาไทยเสร็จสิ้น จำนวน {len(translated_goldens)} เคส")



ขั้นตอนที่ 1: กำลังสร้างชุดข้อมูลเป็นภาษาอังกฤษจากหลายไฟล์...


[Confident AI Synthesizer Log] WARNING: Filtering not applied: Not enough chunks in smallest document

[Confident AI Synthesizer Log] SUCCESS: Successfully deleted: .vector_db

[Confident AI Synthesizer Log] SUCCESS: Context Construction: Utilizing 15 out of 22 chunks.

สร้างชุดข้อมูลภาษาอังกฤษเสร็จสิ้น จำนวน 36 เคส

ขั้นตอนที่ 2: กำลังแปลชุดข้อมูลเป็นภาษาไทย...
  -> กำลังแปลเคสที่ 1/36...
  -> กำลังแปลเคสที่ 2/36...
  -> กำลังแปลเคสที่ 3/36...
  -> กำลังแปลเคสที่ 4/36...
  -> กำลังแปลเคสที่ 5/36...
  -> กำลังแปลเคสที่ 6/36...
  -> กำลังแปลเคสที่ 7/36...
  -> กำลังแปลเคสที่ 8/36...
  -> กำลังแปลเคสที่ 9/36...
  -> กำลังแปลเคสที่ 10/36...
  -> กำลังแปลเคสที่ 11/36...
  -> กำลังแปลเคสที่ 12/36...
  -> กำลังแปลเคสที่ 13/36...
  -> กำลังแปลเคสที่ 14/36...
  -> กำลังแปลเคสที่ 15/36...
  -> กำลังแปลเคสที่ 16/36...
  -> กำลังแปลเคสที่ 17/36...
  -> กำลังแปลเคสที่ 18/36...
  -> กำลังแปลเคสที่ 19/36...
  -> กำลังแปลเคสที่ 20/36...
  -> กำลังแปลเคสที่ 21/36...
  -> กำลังแปลเคสที่ 22/36...
  -> กำลังแปลเคสที่ 23/36...
  -> กำลังแปลเคสที่ 24/36...
  -> กำลังแปลเคสที่ 25/36...
  -> กำลังแปลเคสที่ 26/36...
  -> กำลังแปลเคสที่ 27/36...
  -> กำลังแปลเคสที่ 28/36...
  -> กำลังแปลเคสที่ 29/36...
  -> กำลังแปลเคสที่ 30/36...
  -> กำลังแปลเคสที่ 31/36...
  -> กำลังแปลเคส

In [7]:
translated_goldens

[{'input': 'เวิร์กช็อป TFEX มีเป้าหมายอะไรในการใช้เทคนิค Bollinger Band Market Breadth?',
  'expected_output': 'เวิร์กช็อป TFEX มีเป้าหมายที่จะบรรลุอัตราการชนะ 100% โดยใช้เทคนิค Bollinger Band Market Breadth เพื่อระบุจุดกลับตัวของแนวโน้มและจับจุดต่ำสุดของตลาดอย่างมีประสิทธิภาพ',
  'retrieval_context': None},
 {'input': 'เวิร์กช็อปของ Investic สอนอะไรโดยใช้ Analytics Studio และทรัพยากรเพิ่มเติม?',
  'expected_output': 'เวิร์กช็อปของ Investic สอนทักษะการปฏิบัติสำหรับการซื้อขายและการลงทุนในตลาดต่าง ๆ โดยใช้ Investic Analytics Studio และทรัพยากรเพิ่มเติม ผู้เข้าร่วมจะได้เรียนรู้เทคนิคขั้นสูง เช่น การระบุจุดกลับตัวของแนวโน้มด้วยกลยุทธ์ Bollinger Band Panic Sell การลงทุนในสกุลเงินดิจิทัลโดยเน้นการเลือกเหรียญที่มีศักยภาพสูง และการเชี่ยวชาญทฤษฎี Elliott Wave เพื่อใช้ประโยชน์จากคลื่นที่ทรงพลังที่สุดเพื่อผลกำไรสูงสุด เวิร์กช็อปนี้ออกแบบมาเพื่อให้การเรียนรู้เชิงลึกและประสบการณ์จริงกับเครื่องมือเหล่านี้ ช่วยให้ผู้เข้าร่วมจัดการพอร์ตโฟลิโอด้วยความมั่นใจและเข้าถึงตลาดอย่างมีกลยุทธ์ เช่น TFEX, Crypto

In [9]:
# ===================================================================
# ขั้นตอนที่ 3: บันทึกชุดข้อมูลภาษาไทยเป็นไฟล์ JSON
# ===================================================================
print("\nขั้นตอนที่ 3: กำลังบันทึกชุดข้อมูลภาษาไทย...")
output_file_path = "data/evaluation/golder_dataset_v2.json"

os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

with open(output_file_path, "w", encoding="utf-8") as f:
    json.dump(translated_goldens, f, ensure_ascii=False, indent=4)

print(f"บันทึกชุดข้อมูลภาษาไทยเสร็จสิ้นที่: {output_file_path}")


ขั้นตอนที่ 3: กำลังบันทึกชุดข้อมูลภาษาไทย...
บันทึกชุดข้อมูลภาษาไทยเสร็จสิ้นที่: data/evaluation/golder_dataset_v2.json
